In [ ]:
pip install newspaper3k pandas nltk textblob 

In [ ]:
import re
import nltk
import pandas as pd
from nltk import FreqDist
from newspaper import Article
from textblob import TextBlob
from google.colab import files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import requests
from newspaper import fulltext

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
uploaded = files.upload()
uploaded_keys = list(uploaded.keys())

In [17]:
with open(f'./{uploaded_keys[0]}') as txt:
  url_list = txt.read().splitlines()

url_list= [
  'https://www.viajeroscallejeros.com/maravillas-del-mundo/',
  'https://www.viajeroscallejeros.com/lugares-que-visitar-barcelona/'
]

In [5]:
def clean_text(text):
  remove_characters = re.sub('[^\w\s]', '', text.lower())
  tokenized_text = word_tokenize(remove_characters)
  clean_text = ' '.join([word for word in tokenized_text if word not in stopwords.words('spanish')])
  return clean_text

def most_relevant_kws(text):
  only_nouns = [w for (w, pos) in TextBlob(text).pos_tags if pos[0] == 'N']
  most_repeated_kws = list(FreqDist(only_nouns))
  return most_repeated_kws

In [ ]:
urls_text = []

for url in url_list:
  html = requests.get(url).text
  text = fulltext(html)
  text_clean = clean_text(text)
  urls_text.append({
      "Url": url,
      "Text": text_clean
  })

In [12]:
urls_df = pd.DataFrame.from_dict(urls_text)

In [ ]:
frecuency = []

for text in urls_df['Text']:
  words_frecuency = most_relevant_kws(text)
  frecuency.append(','.join(words_frecuency[0:6]))

urls_df['Frecuency'] = frecuency

In [ ]:
urls_df.drop('Text', axis=1, inplace=True)
urls_df.to_csv('frecuency-urls.csv')
files.download('frecuency-urls.csv')